In [ ]:
### https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b
    
#!/usr/bin/env python3
import math 
import numpy as np
import json
import pandas as pd
import random
import time
import datetime
import re
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F 
from transformers import BertModel
from torch.utils.data import DataLoader, Dataset, TensorDataset,random_split, RandomSampler, SequentialSampler
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report
from torch.utils.data import IterableDataset
from torch.utils.data import DataLoader, Dataset, TensorDataset,random_split, RandomSampler, SequentialSampler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(2)
torch.cuda.current_device()


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def report_average(report_list):
    r_list = list()
    for report in report_list:
        splited = [' '.join(x.split()) for x in report.split('\n\n')]
        header = [x for x in splited[0].split(' ')]
        data = np.array(splited[1].split(' ')).reshape(-1, len(header) + 1)
        data = np.delete(data, 0, 1).astype(float)
        df = pd.DataFrame(data, columns=header)
        r_list.append(df)
    tmp = pd.DataFrame()
    for df in r_list:
        tmp = tmp.add(df, fill_value=0)           
    report_ave =  tmp/len(r_list)
    return(report_ave)

def initialize_model(dataloader, epochs=4):
    bert_classifier = ourBertBinaryClassifier()
    bert_classifier.cuda()
    optimizer = AdamW(bert_classifier.parameters(),lr=0.1,eps=1e-8)
    total_steps = len(dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

class CustomDataset(Dataset):
    def __init__(self,filename,name):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        if name == 'covid':
            self.df = pd.read_csv(filename,delimiter='\t',encoding='utf-8')
            #self.train_df = pd.read_csv("/home/joao/COVID19Tweet-master/train.tsv",delimiter='\t',encoding='utf-8')  
            #self.val_df = pd.read_csv("/home/joao/COVID19Tweet-master/valid.tsv",delimiter='\t',encoding='utf-8')   
            self.df = self.df.rename(columns={'Text': 'sentence'})
            self.df = self.df.rename(columns={'Label': 'label'})
            self.df['label'].replace('INFORMATIVE', 1)
            self.df['label'] = self.df['label'].replace('INFORMATIVE', 1)
            self.df['label'].replace('UNINFORMATIVE', 0)
            self.df['label'] = self.df['label'].replace('UNINFORMATIVE', 0)
        if name == 'crisislext26':
            self.usr_features = pd.DataFrame(columns=['tweet_id','usr_id','usr_vrf','num_followers','num_friends','num_tweets'])
            self.listOfSeries = []
            with open("/home/joao/crisisLexT26.json") as json_file:
                for line in json_file:
                    data = json.loads(line)
                    tweet_id  = data['id_str']
                    user_obj = data['user']
                    usr_id = user_obj['id_str']
                    usr_vrf = int(user_obj['verified'])#.astype(int)
                    num_followers = user_obj['followers_count']
                    if num_followers != 0:
                        num_followers = math.log10(num_followers)
                    num_friends = user_obj['friends_count']
                    if num_friends != 0:
                        num_friends = math.log10(num_friends)
                    num_tweets = user_obj['statuses_count']
                    if num_tweets != 0:
                        num_tweets = math.log10(num_tweets)
                    self.listOfSeries.append(pd.Series([int(tweet_id), usr_id, usr_vrf, num_followers, num_friends, num_tweets ], index=self.usr_features.columns ))
            self.usr_features = self.usr_features.append(self.listOfSeries , ignore_index=True)
            self.usr_features = self.usr_features.astype(int)
            self.df = pd.read_csv("/home/joao/crisisLexT26.csv", encoding='utf-8')
            self.df = self.df.rename(columns={'Tweet ID': 'tweet_id'})
            self.df = self.df.drop([' Information Source', ' Information Type' ], axis=1)
            #Relabelling the columns titles to remove white spaces
            self.df = self.df.rename(columns={' Tweet Text': 'sentence'})
            self.df = self.df.rename(columns={' Informativeness': 'label'})
            self.fd = pd.DataFrame(columns=list(['sentence', 'usr_vrf', 'num_followers', 'num_friends', 'num_tweets', 'label']))
            series = []
            for index, row in self.usr_features.iterrows():
                tweet_id = row['tweet_id']
                usr_vrf, num_followers, num_friends, num_tweets = row['usr_vrf'],row['num_followers'],row['num_friends'],row['num_tweets']
                idx = self.df[self.df['tweet_id']==tweet_id].index.values.astype(int)[0]
                tweet_id, sentence, label = self.df.loc[idx]
                series.append(pd.Series([ sentence , usr_vrf, num_followers, num_friends, num_tweets, label ],index=list(['sentence', 'usr_vrf', 'num_followers', 'num_friends', 'num_tweets', 'label'])))                
            self.fd = self.fd.append(series,ignore_index = True )
            self.df = self.fd
            self.df = self.df.reset_index(drop=True)                             
            self.df = self.df[self.df.label!= 'Not related']
            self.df = self.df[self.df.label!= 'Not applicable']
            self.df['label'].replace('Related and informative', 1)
            self.df['label'] = self.df['label'].replace('Related and informative', 1)
            self.df['label'].replace('Related - but not informative', 0)
            self.df['label'] = self.df['label'].replace('Related - but not informative', 0)
            self.df = self.df.reset_index(drop=True)                       
        if name == 'crisislext6':
            self.df = pd.read_csv("/home/joao/crisisLexT6.csv", encoding='utf-8')
            self.df = self.df.rename(columns={' tweet': 'sentence'})
            self.df = self.df.rename(columns={' label': 'label'})
            self.df['label'].replace('on-topic', 1)
            self.df['label'] = self.df['label'].replace('on-topic', 1)
            self.df['label'].replace('off-topic', 0)
            self.df['label'] = self.df['label'].replace('off-topic', 0)
        self.df = self.df[['sentence','label']]
        self.df['nchars'] = self.df['sentence'].str.len()
        self.df['nwords'] = self.df['sentence'].str.split().str.len()
        self.df['bhash'] = self.df["sentence"].str.contains(pat = '#',flags=re.IGNORECASE, regex = True).astype(int) 
        self.df['nhash'] = self.df["sentence"].str.count('#') 
        self.df['blink']  = self.df["sentence"].str.contains(pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', flags=re.IGNORECASE, regex = True) .astype(int)
        self.df['nlink'] = self.df["sentence"].str.count(pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', flags=re.IGNORECASE) 
        self.df['bat'] = self.df["sentence"].str.contains(pat = '@',flags=re.IGNORECASE, regex = True).astype(int) 
        self.df['nat'] = self.df["sentence"].str.count(pat = '@') 
        self.df['rt'] = self.df["sentence"].str.contains(pat = '@rt|rt@',flags=re.IGNORECASE, regex = True).astype(int) 
        #df['phone'] = df["sentence"].str.contains(pat = '\(?([0-9]{3})\)?([ .-]?)([0-9]{3})\2([0-9]{4})',flags=re.IGNORECASE, regex = True).astype(int) 
        self.df['dlex'] = self.df["sentence"].apply(self.lexical_diversity)
        self.df["sentence"] = self.df["sentence"].str.lower()
        ## List of  US slangs.
        slangList = ['ASAP','BBIAB','BBL','BBS','BF','BFF','BFFL','BRB','CYA','DS','FAQ','FB','FITBLR','FLBP','FML','FTFY','FTW','FYI','G2G','GF','GR8','GTFO','HBIC','HML','HRU','HTH','IDK','IGHT','IMO','IMHO','IMY','IRL','ISTG','JK','JMHO','KTHX','L8R','LMAO','LMFAO','LMK','LOL','MWF','NM','NOOB','NP','NSFW','OOAK','OFC','OMG','ORLY','OTOH','RN','ROFL','RUH','SFW','SOML','SOZ','STFU','TFTI','TIL','TMI','TTFN','TTYL','TWSS','U','W/','WB','W/O','WYD','WTH','WTF','WYM','WYSIWYG','Y','YMMV','YW','YWA']
        slangList = [x.lower() for x in slangList]
        #happy emojis
        happy_emojis = [':\)', ';\)', '\(:']
        #sad emojis
        sad_emojis = [':\(', ';\(', '\):']
        punctuation = ['.',',','...','?','!',':',';']    
        #','-','+','*','_','=','/','','%',' &','{','}','[',']','(',')','
        #Checks if the sentence contains slang
        mask = self.df.iloc[:, 0].str.contains(r'\b(?:{})\b'.format('|'.join(slangList)))
        df1 = self.df[~mask]
        self.df['slang'] = mask.astype(int) 
        #Checks if the sentence contains happy emojis
        mask = self.df.iloc[:, 0].str.contains(r'\b(?:{})\b'.format('|'.join(happy_emojis)), regex = True)
        df1 = self.df[~mask]
        self.df['hemojis'] = mask.astype(int) 
        #Checks if the sentence contains happy emojis
        mask = self.df.iloc[:, 0].str.contains(r'\b(?:{})\b'.format('|'.join(sad_emojis)), regex = True)
        df1 = self.df[~mask]
        self.df['semojis'] = mask.astype(int) 
        self.hand_features =  self.df[['nchars', 'nwords','bhash','nhash','blink','nlink','bat','nat','rt','slang','dlex']]
        self.hand_features_DF = pd.DataFrame(self.hand_features)
        #################
        self.df['sentence'] = self.df['sentence'].str.replace(r'http(\S)+', r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'http(\S)+', r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'http ...', r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'@[\S]+',r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'_[\S]?',r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'[ ]{2, }',r' ')
        self.df['sentence'] = self.df['sentence'].str.replace(r'&amp;?',r'and')
        self.df['sentence'] = self.df['sentence'].str.replace(r'&lt;',r'<')
        self.df['sentence'] = self.df['sentence'].str.replace(r'&gt;',r'>')
        self.df['sentence'] = self.df['sentence'].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
        self.df['sentence'] = self.df['sentence'].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')
        self.df['sentence'] = self.df['sentence'].str.lower()
        self.df['sentence'] = self.df['sentence'].str.strip()
        self.sentences = self.df['sentence']
        self.labels = self.df['label'].values
        self.maxlen = 0
        if name == 'covid':
            self.maxlen = 512
        else:
            for sent in self.sentences:
                input_ids = self.tokenizer.encode(sent, add_special_tokens=True)
                self.maxlen = max(self.maxlen, len(input_ids))
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        sentence = self.df.loc[idx, 'sentence']
        label = self.df.loc[idx, 'label']
        #h_features = self.hand_features_DF.loc[idx,['nchars', 'nwords','bhash','nhash','blink','nlink','bat','nat','rt','slang','dlex','usr_vrf','num_followers','num_friends','num_tweets']]
        h_features = self.hand_features_DF.loc[idx,['nchars', 'nwords','bhash','nhash','blink','nlink','bat','nat','rt','slang','dlex']]
        h_features_tensor = torch.tensor(h_features).to(device)
        tokens = self.tokenizer.tokenize(sentence)
        encoded_dict = self.tokenizer.encode_plus(tokens, add_special_tokens = True, max_length = self.maxlen, pad_to_max_length = True,return_attention_mask = True)
        tokens_ids = encoded_dict['input_ids']
        tokens_ids_tensor = torch.tensor(tokens_ids).to(device) #Converting the list to a pytorch tensor
        attn_mask = encoded_dict['attention_mask']
        attn_mask_tensor = torch.tensor(attn_mask).to(device)
        label_tensor = torch.tensor(label).to(device)
        return tokens_ids_tensor, attn_mask_tensor, label_tensor,h_features_tensor
    def lexical_diversity(self,text):
        return len(set(text.split())) / len(text.split())
    
class ourBertBinaryClassifier(nn.Module):
    def __init__(self, B_in = 768, B_out = 64, H_in = 11, H_out = 32 ):
        super(ourBertBinaryClassifier, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        C_in = B_out + H_out 
        C_out = 2
        self.linear_B = nn.Linear(B_in, B_out)
        self.linear_H = nn.Linear(H_in, H_out)
        #self.classifier = nn.Linear(C_in, C_out)
        #self.classifier = nn.Sequential(nn.Linear(B_in, B_out),nn.ReLU(),nn.Linear(B_out, C_out))
        self.classifier = nn.Sequential(nn.Linear(B_in, B_out),nn.ReLU(),nn.Linear(H_in,H_out),nn.ReLU(),nn.Linear(C_in, C_out))
    def forward(self, seq, attn_masks, hand_features):
        # Feed input to BERT
        outputs = self.bert_layer(input_ids=seq,attention_mask=attn_masks)
        last_hidden_state_cls = outputs[0][:, 0, :]
        logits_B = self.linear_B(last_hidden_state_cls.float())
        logits_H = self.linear_H(hand_features.float())
        cat_features = torch.cat([logits_B.float(),logits_H.float()], dim=1)
        # Feed input to classifier to compute logits
        logits = self.classifier(cat_features.float())
        return logits   

def train(model, train_dataloader, validation_dataloader=None, epochs=4, evaluation=False):
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)
        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()
        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0
        # Put the model into the training mode
        model.train()
        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels, h_features = tuple(t.to(device) for t in batch)
            # Zero out any previously calculated gradients
            model.zero_grad()
            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask, h_features)
            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()
            # Perform a backward pass to calculate gradients
            loss.backward()
            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()
            # Print the loss values and time elapsed for every 20 batches
            if (step % 100 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 100 batches
                time_elapsed = time.time() - t0_batch
                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")
                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()
        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)
        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            model.eval()
            val_accuracy = []
            val_loss = []
            report_list  = []
            for batch in validation_dataloader:
                b_input_ids, b_attn_mask, b_labels, h_features = tuple(t.to(device) for t in batch)
                b_input_ids = batch[0].to(device)
                with torch.no_grad():
                    logits = model(b_input_ids, b_attn_mask, h_features)
                loss = loss_fn(logits, b_labels)
                val_loss.append(loss.item())
                preds = torch.argmax(logits, dim=1).flatten()
                accuracy = (preds == b_labels).cpu().numpy().mean() * 100
                val_accuracy.append(accuracy)
                probs =  F.softmax(logits)
                probs = probs.detach().cpu().numpy()
                predictions  = np.argmax(probs, axis=1).flatten()
                label_ids = b_labels.to('cpu').numpy()
                labels_flat = label_ids.flatten()
                report = classification_report(labels_flat, predictions)
                report_list.append(report)
            val_loss = np.mean(val_loss)
            val_accuracy = np.mean(val_accuracy)
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
            print("\n")
            print("Training complete!")
            print()
            print(report_average(report_list))
            print()

            
        
def train_and_evalCV(model, dataset, cv=10, epochs=nepochs, batch_size = batch_size):
    training_stats = []
    stats = []
    total_t0 = time.time()
    train_dataloader = DataLoader(dataset,sampler = SequentialSampler(dataset), batch_size = batch_size )
    kf = KFold(n_splits=cv)
    for train_index, test_index in kf.split(dataset): # For each fold...
        train_dataset = torch.utils.data.Subset(dataset,train_index)
        val_dataset  = torch.utils.data.Subset(dataset, test_index)
        train_dataloader = DataLoader(train_dataset, sampler = SequentialSampler(train_dataset),batch_size = batch_size)
        val_dataloader = DataLoader(val_dataset, sampler = SequentialSampler(val_dataset),batch_size = batch_size)    
        model, optimizer, scheduler = initialize_model(epochs=nepochs)
        model.cuda()
        for epoch_i in range(0, nepochs):  # For each epoch...
            #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            t0 = time.time()
            total_train_loss = 0
            model.train()
            for step, batch in enumerate(train_dataloader):
                b_input_ids, b_attn_mask, b_labels, h_features = tuple(t.to(device) for t in batch)
                model.zero_grad()
                logits = model(b_input_ids, b_attn_mask, h_features)
                loss = loss_fn(logits, b_labels)
                batch_loss += loss.item()
                total_train_los += loss.item()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
            avg_train_loss = total_train_loss / len(train_dataloader)
            training_time = format_time(time.time() - t0)
            training_stats.append({'epoch': epoch_i + 1,'Training Loss': avg_train_loss,'Training Time': training_time})
            print({'epoch': epoch_i + 1,'Training Loss': avg_train_loss,'Training Time': training_time})
#    sss = StratifiedShuffleSplit(n_splits=cv, test_size=0.5, random_state=0)
#    skf = StratifiedKFold(n_splits=cv)
        predictions , true_labels = [], []
        model.eval()
        for step, batch in enumerate(val_dataloader):
            b_input_ids, b_attn_mask, b_labels, h_features = tuple(t.to(device) for t in batch)
            with torch.no_grad():
                logits = model(b_input_ids, b_attn_mask, h_features)
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()
            labels_flat = label_ids.flatten()
            true_labels = np.concatenate((true_labels,labels_flat))
            predictions = np.concatenate((predictions,pred_flat))
        stats.append(precision_recall_fscore_support(true_labels, predictions, average='macro'))
        print(precision_recall_fscore_support(true_labels, predictions, average='macro'))
        print(classification_report(true_labels,predictions))
    print(stats)
    
    
 
train_dataset =  CustomDataset("/home/joao/COVID19Tweet-master/train.tsv","covid")
val_dataset =  CustomDataset("/home/joao/COVID19Tweet-master/valid.tsv","covid")
datasets = [train_dataset,val_dataset]
dataset = torch.utils.data.ConcatDataset(datasets)

dataset =  CustomDataset(None,"crisislext6")

dataset =  CustomDataset(None,"crisislext26")


train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


batch_size = 8
train_dataloader = DataLoader(train_dataset,sampler = SequentialSampler(train_dataset), batch_size = batch_size )
validation_dataloader = DataLoader(val_dataset,sampler = SequentialSampler(val_dataset),batch_size = batch_size)


#complete data set
dataloader = DataLoader(dataset,sampler = SequentialSampler(dataset), batch_size = batch_size )



set_seed(42)    # Set seed for reproducibility
our_bert_classifier, optimizer, scheduler = initialize_model(dataloader,epochs=4)
train(our_bert_classifier, dataloader, validation_dataloader, epochs=4, evaluation=False)



train_and_evalCV(our_bert_classifier, dataset, cv=10, epochs=10, batch_size = 8)
